In [ ]:
%%bash
# CBGA carboxylate (docked conformer)
antechamber \
    -fi pdb -i ../structures/CBGA.COO.DOCK.1.61.pdb \
    -fo mol2 -o ../structures/CBGA.COO.gaff.mol2 \
    -rn LIG -c bcc -at gaff2 -j 4 -pf y \
    -nc -1
parmchk2 \
    -i ../structures/CBGA.COO.gaff.mol2 \
    -f mol2 -o CBGA.COO.frcmod -s 2

In [ ]:
%%bash
# FAD net formal charge (-2) pH 5.0
antechamber \
    -fi pdb -i ../structures/FAD.pdb \
    -fo mol2 -o ../structures/FAD.gaff.mol2 \
    -rn FAD -c bcc -at gaff2 -j 4 -pf y \
    -nc -2
parmchk2 \
    -i ../structures/FAD.gaff.mol2 \
    -f mol2 -o FAD.frcmod -s 2

In [ ]:
%%bash
# Complex assembly
tleap -f complex.in
# Sanity check
ambpdb -p complex.prmtop -c complex.inpcrd > complex.pdb

In [ ]:
%%bash
# Solvent minimization
sander -O \
    -i em.water.in -o em.water.log -inf /dev/null \
    -p complex.prmtop -c complex.inpcrd \
    -r em.water.rst -ref complex.inpcrd
# Sanity check
ambpdb -p complex.prmtop -c em.water.rst > em.water.pdb

In [ ]:
%%bash
# All atom minimization
sander -O \
    -i em.emin.in -o em.emin.log -inf /dev/null \
    -p complex.prmtop -c em.water.rst -ref em.water.rst \
    -x em.emin.nc -r em.emin.rst 
# Sanity check
ambpdb -p complex.prmtop -c em.emin.rst > em.min.pdb

In [ ]:
%%bash
# Graded initial heating
sander -O \
    -i em.heat.in -o em.heat.log -inf /dev/null \
    -p complex.prmtop -c em.emin.rst \
    -x em.heat.nc -r em.heat.rst 
# Sanity check
ambpdb -p complex.prmtop -c em.emin.rst > em.heat.pdb

In [ ]:
%%bash
# Restrained equilibration 
sander -O \
    -i em.equi.in -o em.equi.log -inf /dev/null \
    -p complex.prmtop -c em.heat.rst -ref em.heat.rst \
    -x em.equi.nc -r em.equi.rst
# Sanity check
ambpdb -p complex.prmtop -c em.equi.rst > em.equi.pdb